In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup
import sqlite3
import inspect
import re
import time

driver = webdriver.Chrome()
driver.get("https://x.com/")


In [2]:
def go_to_following_page():
    driver.get("https://x.com/")
    time.sleep(10)
    profile = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Profile'][data-testid='AppTabBar_Profile_Link']")
    profile.click()
    time.sleep(5)
    followings = driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-1rtiivn > a[href='/Mehr_mihan/following']")
    followings.click()
    time.sleep(5)

In [3]:
def go_down(n):
    try:
        # Create ActionChains object
        actions = ActionChains(driver)

        # Scroll down by simulating "Page Down" key press
        for _ in range(n):  # Adjust the range for the number of scrolls needed
            driver.execute_script("window.scrollBy(0, window.innerHeight);")
            time.sleep(4)  # Adjust the pause as needed to allow content to load

        return True
    except Exception as e:
        print(e)
        return False


In [4]:
def following_list():
    try:
        return driver.find_elements(By.CSS_SELECTOR, "div[data-testid='cellInnerDiv']")
    except:
        return False

In [5]:
def insert_user_name_to_table(text):
    conn = sqlite3.connect('following.db')
    cursor = conn.cursor()
    
    # Check if the text already exists in the table
    cursor.execute('''
    SELECT COUNT(*)
    FROM following_table
    WHERE user_name = ?
    ''', (text,))
    
    # Fetch the result
    count = cursor.fetchone()[0]    
    
    # If count is 0, the text does not exist, so insert it
    if count == 0:    
        cursor.execute('''
        INSERT INTO following_table (user_name)
        VALUES (?)
        ''', (text,))
        conn.commit()
        conn.close()

In [6]:
def delete_user_name_from_table(text):
    # Connect to SQLite database
    conn = sqlite3.connect('following.db')
    
    # Create a cursor object using the cursor() method
    cursor = conn.cursor()
    
    # SQL command to delete the text from the table
    cursor.execute('''
    DELETE FROM following_table
    WHERE user_name = ?
    ''', (text,))
    
    # Check if any row was deleted
    if cursor.rowcount == 0:
        print(f"Text '{text}' does not exist in the table.")
    else:
        print(f"Text '{text}' deleted successfully.")
    
    # Commit the changes
    conn.commit()
    
    # Close the connection
    conn.close()


In [7]:
import sqlite3

def count_table_records():
    # Connect to SQLite database
    conn = sqlite3.connect('following.db')
    
    # Create a cursor object using the cursor() method
    cursor = conn.cursor()
    
    # SQL command to delete the text from the table
    cursor.execute('SELECT COUNT(*) FROM following_table')
    
        # Fetch the result
    count = cursor.fetchone()[0]

    # Close the connection
    conn.close()
    
    # Return the count
    return count

# Example usage
record_count = count_table_records()
print(f"The number of records in the table is: {record_count}")

The number of records in the table is: 0


In [8]:
def add_user_name_of_following_to_db(list_of_followings):
    try:
        # Iterate through each list item
        for following in list_of_followings:
            try:
                user_name = following.find_element(By.CSS_SELECTOR, "a.css-175oi2r.r-1wbh5a2.r-dnmrzs.r-1ny4l3l.r-1loqt21").get_attribute("href")
                insert_user_name_to_table(user_name)
            except:
                {}
        return True
    except:
        return False


In [9]:
def is_the_true_page():
    try:
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/verified_followers'][role='tab']")
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/followers'][role='tab']")
        driver.find_element(By.CSS_SELECTOR, "div.css-175oi2r.r-14tvyh0.r-cpa5s6.r-16y2uox > a[href='/Mehr_mihan/following'][role='tab'][aria-selected='true']")
        return True
    except:
        return False

In [10]:
def collect_all_use_name_of_following():
    try:
        if not is_the_true_page():
            go_to_following_page()

        go_down(1)            
        list_of_followings = following_list()
        if add_user_name_of_following_to_db(list_of_followings):
            return True
        else:
            return False
        
    except:
        return False

In [11]:
if __name__ == '__main__':
    go_to_following_page()
    
    while True:
        if collect_all_use_name_of_following():
            time.sleep(5)
        else:
            driver.refresh()
            go_to_following_page()